---
## Fehlende Werte mit PANDAS

In [1]:
import numpy as np
import pandas as pd
from io import StringIO
from sklearn.preprocessing import LabelEncoder

In [2]:
csv_data = \
    '''A,B,C,D
        1.0,2.0,3.0,4.0
        5.0,6.0,,8.0
        10.0,11.0,12.0,
    '''
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


### Fehlende Werte finden
`1. isna()`       : Liefert `True` zurück, in den Zellen, die n/a haben<br>
`2. isna().sum()` : Gibt je Spalte die Summe als `int` zurück

In [3]:
df.isna()

,A,B,C,D
0,False,False,False,False
1,False,False,True,False
2,False,False,False,True


In [4]:
df.isna().sum()

A    0
B    0
C    1
D    1
dtype: int64

### Die meisten SKLEARN Bibliotheken können mit Pandas arbeiten, sind aber auf NUMPY spezialisiert 
Sprich man sollte möglichst Numpy Arrays übergeben, die man via `df.values` erhalten kann

In [5]:
df.values

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

In [6]:
type(df.values)

numpy.ndarray

---
### 1. fehlende Features (Merkmale) oder Zeilen (Instanzen) komplett entfernen

In [7]:
# Spalte komplett entfernen
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [8]:
# Zeile komplett entfernen
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [9]:
# Nur Zeilen löschen, in denen alle Spalten N/A enthalten -> gibt ganzes Array zurück, weil keine Zeile durchgängig nur N/A enthält
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [10]:
# Treshhold verlangt eine Mindestanzahl von reelen Zahlen in der Zeile --> löscht beide Zeilen, in denen das nicht der Fall ist
df.dropna(thresh=4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [11]:
# Subset löscht nur Zeilen, in denen N/A in einer definierten Spalte enthalten ist
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


---
## IMPUTATION TECHNICS

### 1. Imputation mit gesamt-DF MEAN() Wert

In [12]:
df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


### 2. Eine Spalte die N/A mit dem MEAN() derselben Spalte auffüllen

In [13]:
df['C'] = df['C'].fillna(df['C'].mean())
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,NaN


### 3. Handhabung kategorialer Variablen in PANDAS
Man muss zwischen `a) ordinalen` und `b) nominalen` Merkmalen unterscheiden:<br>
<br>
`ad a)` natürliche Reihenfolge, können sortiert werden - aber Abstände zwischen den einzelnen Werten sind nicht quantifizierbar (es kann nicht mit ihnen “normal gerechnet” werden. Beispiel: Schulnoten, Präferenzrangfolgen, etc.<br>
<br>
`ad b)` Daten, die in keinerlei natürliche Reihenfolge gebracht werden können – beispielsweise um das Geschlecht, die Haarfarbe oder die Telefonnummer

In [28]:
df = pd.DataFrame([
    ["green", "M", 10.1, "CLASS2"],
    ["red",   "L", 13.5, "CLASS1"],
    ["blue", "XL", 15.3, "CLASS2"],
])
df.columns = ["color", "size", "price", "classLabel"]
df

,color,size,price,classLabel
0,green,M,10.1,CLASS2
1,red,L,13.5,CLASS1
2,blue,XL,15.3,CLASS2


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   color       3 non-null      object 
 1   size        3 non-null      object 
 2   price       3 non-null      float64
 3   classLabel  3 non-null      object 
dtypes: float64(1), object(3)
memory usage: 224.0+ bytes


### a) ORDINALE MERKMALE
Um zu gewährleisten, dass die Algorithmen die Daten korrekt verarbeiten können, müssen wir die kategorialen Variablen in INT-Werte umwandeln.<br> 
Es gibt keine Funktion, die das macht, daher müssen wir das von Hand erledigen.<br>
Wir gehen davon aus, dass wir die numerischen Unterschiede zwischen den Merkmalen kennen -> `XL = L + 1 = M + 2`

In [30]:
size_mapping = {
    'M':  1,
    'L':  2,
    'XL': 3,
    }
df["size"] = df["size"].map(size_mapping)
df

,color,size,price,classLabel
0,green,1,10.1,CLASS2
1,red,2,13.5,CLASS1
2,blue,3,15.3,CLASS2


Wenn wir die Spalte `"size"` später wieder zurück umwandeln wollen, dann können wir ebenfalls ebenfalls über ein Dictionary machen

In [31]:
inv_size_mapping = {v: k for k, v in size_mapping.items()}
print(inv_size_mapping)
_ = df.copy()
_["size"] = df["size"].map(inv_size_mapping)
_

{1: 'M', 2: 'L', 3: 'XL'}


,color,size,price,classLabel
0,green,M,10.1,CLASS2
1,red,L,13.5,CLASS1
2,blue,XL,15.3,CLASS2


### b) NORMINALE MERKMALE
Viele Bibliotheken machen es erforderliche, dass die Klassenbezeichnungen als INT-Zahlen codiert sind.<br>
Bei norminalen Merkmalen hat die Reihenfolge oder dergleichen keine Relevanz, wir können also einfach durchnummerieren

In [32]:
class_mapping = {label: idx for idx, label in enumerate(np.unique(df["classLabel"]))}
print(class_mapping)
df["classLabel"] = df["classLabel"].map(class_mapping)
df

{'CLASS1': 0, 'CLASS2': 1}


,color,size,price,classLabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


Umkehrung wie folgt:

In [37]:
inv_class_mapping = {v: k for k, v in class_mapping.items()}
print(inv_class_mapping)
df["classLabel"] = df["classLabel"].map(inv_class_mapping)
df

{0: 'CLASS1', 1: 'CLASS2'}


,color,size,price,classLabel
0,green,1,10.1,CLASS2
1,red,2,13.5,CLASS1
2,blue,3,15.3,CLASS2


### EINE WEITER MÖGLICHKEIT WÄREN DIE LABEL-ENCODER VON SKLEARN

In [38]:
class_le = LabelEncoder()
y = class_le.fit_transform(df["classLabel"].values)
y

array([1, 0, 1])

In [39]:
class_le.inverse_transform(y)

array(['CLASS2', 'CLASS1', 'CLASS2'], dtype=object)

## ONE-HOT-ENCODING DER NOMINALEN MERKMALE